### 1. 图像数据处理

In [17]:
from PIL import Image
from torchvision import transforms as T
import timm

In [8]:
img1 = Image.open('./imgs/cat.jpg')
img2 = Image.open('./imgs/cat2.jpg')
img3 = Image.open('./imgs/face.jpg')

In [21]:
trans = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=timm.data.IMAGENET_DEFAULT_MEAN, std=timm.data.IMAGENET_DEFAULT_STD)
])

In [26]:
t_img1 = trans(img1).unsqueeze(0)
t_img2 = trans(img2).unsqueeze(0)
t_img3 = trans(img3).unsqueeze(0)

### 2. 加载模型

In [34]:
from torchvision import models
import torch
from torch import nn

In [12]:
model = models.resnet152(pretrained=True)

In [14]:
layer = model._modules['avgpool']

In [30]:
model.eval()
with torch.no_grad():
    feature_vec3 = torch.zeros(2048)
    def copy(m, i, o):
        print(o.shape)
        feature_vec3.copy_(o.reshape(o.shape[1]))
    h = layer.register_forward_hook(copy)
    model(t_img3)   # forward
    h.remove()

torch.Size([1, 2048, 1, 1])


In [33]:
feature_vec3

tensor([0.1978, 0.4018, 0.0321,  ..., 0.0981, 0.2331, 0.2016])

### 3. 输出处理

In [35]:
cos_sim = nn.CosineSimilarity(dim=1)

In [37]:
cos_sim(feature_vec.unsqueeze(0), feature_vec2.unsqueeze(0))

tensor([0.6978])

In [38]:
cos_sim(feature_vec.unsqueeze(0), feature_vec3.unsqueeze(0))

tensor([0.5045])